## Warning: 

This notebook wipes your Experiment history clean. Use only if you want to start fresh. You will lose all experiments, trials, trial components and hosted endpoints if you run this notebook.

You have been warned.

In [ ]:
# !pip install -q --upgrade pip
# !pip install -q sagemaker smdebug awscli sagemaker-experiments --upgrade

In [ ]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

In [ ]:
import time
import boto3
sess = boto3.Session()
sm = sess.client('sagemaker')

In [ ]:
def cleanup_experiment(experiment):
    for trial_summary in experiment.list_trials():
        try:
            trial = Trial.load(sagemaker_boto_client=sm, trial_name=trial_summary.trial_name)
            for trial_component_summary in trial.list_trial_components():
                tc = TrialComponent.load(
                    sagemaker_boto_client=sm,
                    trial_component_name=trial_component_summary.trial_component_name)
                try:
                    trial.remove_trial_component(tc)
                    print(f'Deleting trial component: {tc.trial_component_name}')
                    tc.delete()
                except:
                    time.sleep(0.4)
                    continue
            print(f'Deleting trial: {trial.trial_name}')
            trial.delete()
        except:
            continue
    print(f'Deleting experiment: {experiment.experiment_name}')
    experiment.delete()

#### List of existing experiments

In [ ]:
exp_names = [exp.experiment_name for exp in Experiment.list(sagemaker_boto_client=sm)]
print('Current experiments:')
exp_names

In [ ]:
for exp in exp_names:
    cleanup_experiment(Experiment.load(exp))

#### Delete unassigned trial components

In [ ]:
for tc in TrialComponent.list(sagemaker_boto_client=sm):
    try:
        sm.delete_trial_component(TrialComponentName=tc.trial_component_name)
        print(f'Deleted: {tc.trial_component_name}')
        time.sleep(0.4)
    except:
        continue

#### Delete all endpoints

In [ ]:
for en in sm.list_endpoints()['Endpoints']:
    try:
        print(en['EndpointName'])
        sm.delete_endpoint(EndpointName=en['EndpointName'])
        time.sleep(0.4)
    except:
        continue